## Testing Code for Social Role Study Task

In [1]:
import csv, os, random, re
from random import shuffle
import numpy as np
import pandas as pd
from IPython.display import HTML
import scipy as sp
from scipy import stats
from scipy.stats import zscore
from matplotlib import pyplot as plt
from IPython.display import Markdown, display
import glob
import collections 
from collections import Counter
import itertools
from itertools import *
import copy

In [2]:
raw = pd.read_csv('/Users/Elyssa/Work/Thornton/Expts/Social_Role/Task/Stimuli/scenarios_final.csv')

In [3]:
raw.head()

,mentalstate,scenario,new_num
0,alarm,waking up to a screaming baby,1
1,alarm,hearing a coyote while camping,2
2,alarm,hearing crashing plates,3
3,alarm,seeing a bear while hiking,4
4,alarm,getting a nasty email from the boss,5


### For a given state, each of the four target people should get a different set of eight scenario pairs, drawn from the common set of 16 scenarios for that state

- first, randomly separate the scenarios into two lists. 
- pair each scenario with its corresponding number in the other list.
- proceed down the two lists in parallel in this way to get the 8 pairings you want. 
- To get the next pairing, you simply shift list2 one index relative to list1. That is, remove the last value from list2, and then prepend that value to the beginning of list2. 
- proceed through two lists again tp get a completely non-overlapping set of pairs. 
- Repeat this process twice more, and you will end up with four sets of non-overlapping pairs of the same 16 scenarios.

In [4]:
states = ['alarm', 'attention', 'awareness', 'awe', 'belief', 'cognition',
          'contemplation', 'craziness', 'curiosity', 'desire', 'disarray',
          'distrust', 'embarrassment', 'fatigue', 'laziness', 'opinion',
          'peacefulness', 'pity', 'playfulness', 'relaxation',
          'self-consciousness', 'self-pity', 'skepticism', 'subordination', 'uneasiness']

In [5]:
#Create a non-overlapping list of scenarios for each state, for each target

#start with empty lists for each target
target1 = []
target2 = []
target3 = []
target4 = []
for s in states:
    s = raw[raw['mentalstate'].str.match(s)] #find all rows for given mental states
    s = s.values.tolist() #push those rows to a list
    random.shuffle(s) # shuffle the list
    s1 = s[:8] #create sublist for first 8 values from shuffled list
    s2 = s[8:] #create second sublist for last 8 values
    s3 = s[8:] 
    s3.insert(0,s3[-1]) #taking the last row and prepending to the list
    s4 = s3[:8] 
    s4.insert(0,s4[-1]) #starting with list2, take last row and prepend to list
    s5 = s4[:8]
    s5.insert(0,s5[-1])#starting with list3, take last row and prepend to list

    for index in range(len(s1)):
        pairs1 = [index, s1[index], s2[index]] #pair list1 and list2 based on index to get list for target1
        target1.append(pairs1) #add the pairs to the target1 list
        pairs2 = [index, s1[index], s3[index]]#pair lists based on index to get list for target2
        target2.append(pairs2)#add the pairs to the target2 list
        pairs3 = [index, s1[index], s4[index]]#pair lists based on index to get list for target3
        target3.append(pairs3)#add the pairs to the target3 list
        pairs4 = [index, s1[index], s5[index]]#pair lists based on index to get list for target4
        target4.append(pairs4)#add the pairs to the target4 list

In [6]:
target1

[[0,
  ['alarm', 'getting a nasty email from the boss', 5],
  ['alarm', 'hair catches on fire from birthday candles', 15]],
 [1,
  ['alarm', 'hearing a gun shot', 9],
  ['alarm', 'locking keys inside of a car', 16]],
 [2,
  ['alarm', 'realizing when the car is almost out of fuel', 14],
  ['alarm', 'reading a murderer is running loose', 8]],
 [3,
  ['alarm', 'losing child in a crowd', 12],
  ['alarm', 'waking up to a fire alarm', 6]],
 [4,
  ['alarm', 'seeing a robbery take place', 10],
  ['alarm', 'hearing a hurricane warning', 11]],
 [5,
  ['alarm', 'seeing a car accident happen', 13],
  ['alarm', 'waking up to a screaming baby', 1]],
 [6,
  ['alarm', 'hearing about a possible epidemic', 7],
  ['alarm', 'hearing a coyote while camping', 2]],
 [7,
  ['alarm', 'hearing crashing plates', 3],
  ['alarm', 'seeing a bear while hiking', 4]],
 [0,
  ['attention', "looking deep into someone's eyes", 20],
  ['attention', 'playing in an orchestra', 23]],
 [1,
  ['attention', 'focusing on a conve

In [7]:
# Making the trial lists for each run - saved as 1 list (run_list)
targslist = ['target1', 'target2', 'target3', 'target4']
runNum = 8
trials = [[x,y] for x in targslist for y in states]
run_list = []
for i in range(runNum):
    random.shuffle(trials)
    run_list.append(copy.deepcopy(trials))

In [8]:
run_list[1][1][1]

'desire'

In [9]:
run_list

[[['target3', 'belief'],
  ['target1', 'fatigue'],
  ['target2', 'pity'],
  ['target1', 'self-pity'],
  ['target3', 'distrust'],
  ['target2', 'subordination'],
  ['target1', 'playfulness'],
  ['target3', 'self-consciousness'],
  ['target1', 'relaxation'],
  ['target2', 'distrust'],
  ['target4', 'subordination'],
  ['target1', 'disarray'],
  ['target3', 'contemplation'],
  ['target1', 'cognition'],
  ['target3', 'opinion'],
  ['target1', 'skepticism'],
  ['target2', 'opinion'],
  ['target3', 'awe'],
  ['target4', 'embarrassment'],
  ['target2', 'contemplation'],
  ['target4', 'belief'],
  ['target3', 'subordination'],
  ['target2', 'awareness'],
  ['target4', 'relaxation'],
  ['target1', 'craziness'],
  ['target4', 'distrust'],
  ['target4', 'fatigue'],
  ['target3', 'laziness'],
  ['target3', 'cognition'],
  ['target2', 'skepticism'],
  ['target1', 'pity'],
  ['target3', 'craziness'],
  ['target3', 'pity'],
  ['target4', 'attention'],
  ['target2', 'attention'],
  ['target2', 'embarr

### assign state pairings to particular trials

- read through a list (or set of lists) which specify the target and state of each trial. You can use this information to access the appropriate sets of scenario pairings generated in (1). 
- optimize the balance of scenarios across runs (i.e., we don't want a certain scenario to appear a bunch in run 1, and not at all in run 2). 
    - as you progress through each run, you can keep a running count of how often each scenario has appeared. 
    - In Python, this could take the form of a length 400 list (each element representing one of the 25 x 16 scenarios) initial composed entirely of 0s. Whenever a new pair of scenario is assigned to a trial, the corresponding values in this list would increment. 
    - To assign a pair of scenarios to a given trial, for each of the possible pairs you would check out often each scenario in the pair had already been used in the run. For instance, to continue the example above, on the second trial (if it happened to call for the same state, but a different target) and one pairing might be 94 and 95, and another might be 97 and 98. Now, neither 97 nor 98 have been used, so by pulling those indices in the list you would get 0 and 0, for a grand total of 0. However, in the other pairing, 94 has already been used in the first trial, so you would get 1 and 0 = 1. 
    - You can calculate this value for each remaining pairing (i.e., that's appropriate for the current target+state, and hasn't been used in an earlier run). Then you can select whichever scenario has the lowest value (i.e. has been used the least in that run). When there are ties, you can simply randomly choose among the tied choices. As this procedure is applied across trials, it minimizes the imbalance between how often different scenarios are used. 

In [15]:
d = {'target1':target1, 'target2':target2, 'target3':target3, 'target4':target4}

def getPairs(target):
    for t in run_target:
        if t in d:
            return (d[t])

#Create Counter to track how often a scenario has been used.
counter1 = [0]*400

trialNum = 100       

for run in range(runNum):
    run_target = [i[0] for i in run_list[run]]
    run_state = [i[1] for i in run_list[run]]
    get_targ = getPairs(run_target)

scenarios = [x for x in get_targ if x[1][0] == run_list[1][1][1]]    
# global_sel_ind = [i for i,x in enumerate(d[run_target[0]]) if x[1][2] == scenarios[sel_ind][1][2]]
curr_counts = []


for trial in range(trialNum):
    for pair in scenarios:
        count1 = counter1[pair[1][2]-1]
        count2 = counter1[pair[2][2]-1]
        curr_counts.append(count1 + count2)
        count_min = min(curr_counts)
        min_inds = [i for i,x in enumerate(curr_counts) if x == count_min]
        random.shuffle(min_inds)
        sel_ind = min_inds[0]
        global_sel_ind = [i for i,x in enumerate(d[run_target[0]]) if x[1][2] == scenarios[sel_ind][1][2]]
        #updating the counter
        counter1[scenarios[sel_ind][1][2]]+=1
        counter1[scenarios[sel_ind][2][2]]+=1
#       del d[run_target[0]][global_sel_ind[0]]
        
        

    

IndexError: list index out of range

In [11]:
global_sel_ind = [i for i,x in enumerate(d[run_target[0]]) if x[1][2] == scenarios[sel_ind][1][2]]

NameError: name 'd' is not defined

In [72]:
count_min = min(curr_counts)
min_inds = [i for i,x in enumerate(curr_counts) if x == count_min]

In [73]:
random.shuffle(min_inds)
sel_ind = min_inds[0]

In [74]:
scenarios[sel_ind][1][1]

'watching a child get bullied'

In [75]:
scenarios[sel_ind][2][1]

'starving children in Africa'

In [76]:
#updating the counter
counter1[scenarios[sel_ind][1][2]]+=1
counter1[scenarios[sel_ind][2][2]]+=1

In [45]:
counter1

[0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [13]:
get_targ

[[0,
  ['alarm', 'hearing crashing plates', 3],
  ['alarm', 'losing child in a crowd', 12]],
 [1,
  ['alarm', 'getting a nasty email from the boss', 5],
  ['alarm', 'hearing a gun shot', 9]],
 [2,
  ['alarm', 'seeing a bear while hiking', 4],
  ['alarm', 'realizing when the car is almost out of fuel', 14]],
 [3,
  ['alarm', 'hearing about a possible epidemic', 7],
  ['alarm', 'locking keys inside of a car', 16]],
 [4,
  ['alarm', 'waking up to a screaming baby', 1],
  ['alarm', 'seeing a robbery take place', 10]],
 [5,
  ['alarm', 'reading a murderer is running loose', 8],
  ['alarm', 'waking up to a fire alarm', 6]],
 [6,
  ['alarm', 'seeing a car accident happen', 13],
  ['alarm', 'hair catches on fire from birthday candles', 15]],
 [7,
  ['alarm', 'hearing a coyote while camping', 2],
  ['alarm', 'hearing a hurricane warning', 11]],
 [0,
  ['attention', 'when the test date is announced', 25],
  ['attention', 'playing in an orchestra', 23]],
 [1,
  ['attention', 'closing eyes to try 

In [99]:
run_state

['laziness',
 'cognition',
 'skepticism',
 'disarray',
 'awe',
 'uneasiness',
 'uneasiness',
 'cognition',
 'pity',
 'belief',
 'contemplation',
 'subordination',
 'pity',
 'self-pity',
 'relaxation',
 'self-consciousness',
 'awareness',
 'attention',
 'belief',
 'alarm',
 'awareness',
 'uneasiness',
 'contemplation',
 'attention',
 'alarm',
 'peacefulness',
 'subordination',
 'pity',
 'self-pity',
 'distrust',
 'opinion',
 'opinion',
 'pity',
 'fatigue',
 'peacefulness',
 'awe',
 'desire',
 'playfulness',
 'skepticism',
 'contemplation',
 'subordination',
 'awe',
 'embarrassment',
 'distrust',
 'uneasiness',
 'fatigue',
 'self-consciousness',
 'contemplation',
 'relaxation',
 'craziness',
 'embarrassment',
 'cognition',
 'alarm',
 'fatigue',
 'self-pity',
 'curiosity',
 'disarray',
 'subordination',
 'self-pity',
 'desire',
 'curiosity',
 'embarrassment',
 'distrust',
 'fatigue',
 'disarray',
 'belief',
 'embarrassment',
 'curiosity',
 'desire',
 'disarray',
 'laziness',
 'playfulness

In [90]:
get_state

NameError: name 'get_state' is not defined

# Trying things out below...not directly part of code

In [ ]:
sum([item[] for item in example_list])

In [76]:
# selecting the first scenario in each pair
[i[0] for i in target1]

[['alarm', 'realizing when the car is almost out of fuel', 14],
 ['alarm', 'hearing a hurricane warning', 11],
 ['alarm', 'seeing a car accident happen', 13],
 ['alarm', 'waking up to a fire alarm', 6],
 ['alarm', 'locking keys inside of a car', 16],
 ['alarm', 'hearing about a possible epidemic', 7],
 ['alarm', 'hearing a coyote while camping', 2],
 ['alarm', 'hair catches on fire from birthday candles', 15],
 ['attention', 'hearing instructions before test', 24],
 ['attention', "looking deep into someone's eyes", 20],
 ['attention', 'listening to an engaging speaker', 17],
 ['attention', 'eavesdropping', 26],
 ['attention', 'taking notes in class', 27],
 ['attention', 'hearing a strange sound while camping', 31],
 ['attention', 'tracing a detailed drawing', 32],
 ['attention', 'hearing results of medical examination', 28],
 ['awareness', 'noticing cultural differences', 37],
 ['awareness', 'noticing who is in the room', 41],
 ['awareness', 'noticing chest rise and fall with each brea

In [19]:
#selecting the second scenario in each pair
[i[1] for i in target1]

[['alarm', 'getting a nasty email from the boss', 5],
 ['alarm', 'hearing crashing plates', 3],
 ['alarm', 'seeing a bear while hiking', 4],
 ['alarm', 'hearing a gun shot', 9],
 ['alarm', 'reading a murderer is running loose', 8],
 ['alarm', 'waking up to a screaming baby', 1],
 ['alarm', 'seeing a robbery take place', 10],
 ['alarm', 'losing child in a crowd', 12],
 ['attention', 'when the test date is announced', 25],
 ['attention', 'focusing on a conversation', 21],
 ['attention', 'closing eyes to try to hear a soft sound', 19],
 ['attention', 'drawing a detailed scene', 18],
 ['attention', 'carefully painting edges of a room', 22],
 ['attention', 'hearing voice on a loudspeaker', 30],
 ['attention', 'listening to safety instructions', 29],
 ['attention', 'playing in an orchestra', 23],
 ['awareness', 'sense of being watched', 47],
 ['awareness', 'noticing a new smell', 35],
 ['awareness', 'noticing what people nearby are doing', 40],
 ['awareness', 'listening intently to a strange

#### What we have here for lists target1-target4 are "nested lists"
### in the code below, it's [i[index for which list in that index you want - the first or the second][index for which "column" you want within that list] for i in NameofMasterListPullingFrom]

In [20]:
#pulls the new numbers column
[i[1][2] for i in target1]

[5,
 3,
 4,
 9,
 8,
 1,
 10,
 12,
 25,
 21,
 19,
 18,
 22,
 30,
 29,
 23,
 47,
 35,
 40,
 39,
 45,
 36,
 46,
 34,
 51,
 52,
 59,
 54,
 64,
 60,
 50,
 53,
 75,
 73,
 68,
 77,
 71,
 76,
 70,
 67,
 90,
 85,
 95,
 81,
 86,
 83,
 96,
 89,
 103,
 98,
 100,
 106,
 105,
 99,
 111,
 101,
 116,
 128,
 115,
 123,
 120,
 119,
 127,
 117,
 134,
 141,
 136,
 133,
 140,
 130,
 142,
 135,
 145,
 154,
 152,
 149,
 147,
 155,
 157,
 153,
 162,
 176,
 165,
 172,
 168,
 169,
 164,
 175,
 187,
 184,
 185,
 190,
 188,
 189,
 186,
 181,
 197,
 203,
 202,
 196,
 199,
 205,
 208,
 207,
 219,
 214,
 211,
 210,
 224,
 215,
 220,
 218,
 232,
 240,
 226,
 239,
 231,
 236,
 225,
 230,
 241,
 253,
 248,
 250,
 247,
 249,
 252,
 244,
 260,
 270,
 272,
 265,
 264,
 258,
 261,
 262,
 277,
 274,
 282,
 278,
 280,
 281,
 273,
 276,
 297,
 303,
 296,
 304,
 295,
 298,
 294,
 301,
 309,
 320,
 314,
 312,
 310,
 319,
 313,
 311,
 332,
 322,
 331,
 324,
 329,
 321,
 327,
 328,
 349,
 347,
 352,
 337,
 348,
 345,
 344,
 346,


In [16]:
raw.head()

,mentalstate,scenario,new_num
0,alarm,waking up to a screaming baby,1
1,alarm,hearing a coyote while camping,2
2,alarm,hearing crashing plates,3
3,alarm,seeing a bear while hiking,4
4,alarm,getting a nasty email from the boss,5


In [20]:
states

['alarm',
 'attention',
 'awareness',
 'awe',
 'belief',
 'cognition',
 'contemplation',
 'craziness',
 'curiosity',
 'desire',
 'disarray',
 'distrust',
 'embarrassment',
 'fatigue',
 'laziness',
 'opinion',
 'peacefulness',
 'pity',
 'playfulness',
 'relaxation',
 'self-consciousness',
 'self-pity',
 'skepticism',
 'subordination',
 'uneasiness']

In [227]:
state_list = list(itertools.combinations(states,2))

In [244]:
state_list

[('alarm', 'attention'),
 ('alarm', 'awareness'),
 ('alarm', 'awe'),
 ('alarm', 'belief'),
 ('alarm', 'cognition'),
 ('alarm', 'contemplation'),
 ('alarm', 'craziness'),
 ('alarm', 'curiosity'),
 ('alarm', 'desire'),
 ('alarm', 'disarray'),
 ('alarm', 'distrust'),
 ('alarm', 'embarrassment'),
 ('alarm', 'fatigue'),
 ('alarm', 'laziness'),
 ('alarm', 'opinion'),
 ('alarm', 'peacefulness'),
 ('alarm', 'pity'),
 ('alarm', 'playfulness'),
 ('alarm', 'relaxation'),
 ('alarm', 'self-consciousness'),
 ('alarm', 'self-pity'),
 ('alarm', 'skepticism'),
 ('alarm', 'subordination'),
 ('alarm', 'uneasiness'),
 ('attention', 'awareness'),
 ('attention', 'awe'),
 ('attention', 'belief'),
 ('attention', 'cognition'),
 ('attention', 'contemplation'),
 ('attention', 'craziness'),
 ('attention', 'curiosity'),
 ('attention', 'desire'),
 ('attention', 'disarray'),
 ('attention', 'distrust'),
 ('attention', 'embarrassment'),
 ('attention', 'fatigue'),
 ('attention', 'laziness'),
 ('attention', 'opinion'),


# Need to find a way to randomize the order of the states in state_list (so alarm isn't always on the left)

In [42]:
random.shuffle(state_list)

In [201]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [202]:
chunk = list(chunks(state_list, 60))

In [203]:
df = pd.DataFrame(chunk)

In [206]:
df = pd.DataFrame.transpose(df)

In [207]:
df.columns = ['a','b','c','d','e']

In [208]:
df

,a,b,c,d,e
0,"(attention, embarrassment)","(embarrassment, self-consciousness)","(relaxation, uneasiness)","(cognition, self-pity)","(curiosity, distrust)"
1,"(peacefulness, pity)","(craziness, disarray)","(laziness, uneasiness)","(fatigue, pity)","(alarm, cognition)"
2,"(opinion, subordination)","(desire, skepticism)","(playfulness, subordination)","(laziness, skepticism)","(alarm, laziness)"
3,"(fatigue, subordination)","(desire, laziness)","(cognition, pity)","(awe, self-pity)","(alarm, contemplation)"
4,"(distrust, playfulness)","(awareness, uneasiness)","(desire, pity)","(craziness, skepticism)","(playfulness, skepticism)"
5,"(craziness, laziness)","(laziness, pity)","(awareness, disarray)","(opinion, peacefulness)","(awe, belief)"
6,"(alarm, peacefulness)","(craziness, playfulness)","(awareness, relaxation)","(awareness, opinion)","(belief, curiosity)"
7,"(craziness, self-consciousness)","(belief, fatigue)","(cognition, uneasiness)","(fatigue, self-pity)","(awe, pity)"
8,"(laziness, self-consciousness)","(curiosity, relaxation)","(fatigue, opinion)","(disarray, subordination)","(curiosity, uneasiness)"
9,"(playfulness, self-consciousness)","(awareness, embarrassment)","(opinion, playfulness)","(alarm, distrust)","(curiosity, playfulness)"


In [209]:
df['a']= df['a'].astype(str)

In [210]:
df['b']= df['b'].astype(str)
df['c']= df['c'].astype(str)
df['d']= df['d'].astype(str)
df['e']= df['e'].astype(str)

In [213]:
df['a1'], df['a2'] = df['a'].str.split(',', 1).str
df['b1'], df['b2'] = df['b'].str.split(',', 1).str
df['c1'], df['c2'] = df['c'].str.split(',', 1).str
df['d1'], df['d2'] = df['d'].str.split(',', 1).str
df['e1'], df['e2'] = df['e'].str.split(',', 1).str

In [216]:
df.head()

,a,b,c,d,e,a1,a2,b1,b2,c1,c2,d1,d2,e1,e2
0,"('attention', 'embarrassment')","('embarrassment', 'self-consciousness')","('relaxation', 'uneasiness')","('cognition', 'self-pity')","('curiosity', 'distrust')",('attention','embarrassment'),('embarrassment','self-consciousness'),('relaxation','uneasiness'),('cognition','self-pity'),('curiosity','distrust')
1,"('peacefulness', 'pity')","('craziness', 'disarray')","('laziness', 'uneasiness')","('fatigue', 'pity')","('alarm', 'cognition')",('peacefulness','pity'),('craziness','disarray'),('laziness','uneasiness'),('fatigue','pity'),('alarm','cognition')
2,"('opinion', 'subordination')","('desire', 'skepticism')","('playfulness', 'subordination')","('laziness', 'skepticism')","('alarm', 'laziness')",('opinion','subordination'),('desire','skepticism'),('playfulness','subordination'),('laziness','skepticism'),('alarm','laziness')
3,"('fatigue', 'subordination')","('desire', 'laziness')","('cognition', 'pity')","('awe', 'self-pity')","('alarm', 'contemplation')",('fatigue','subordination'),('desire','laziness'),('cognition','pity'),('awe','self-pity'),('alarm','contemplation')
4,"('distrust', 'playfulness')","('awareness', 'uneasiness')","('desire', 'pity')","('craziness', 'skepticism')","('playfulness', 'skepticism')",('distrust','playfulness'),('awareness','uneasiness'),('desire','pity'),('craziness','skepticism'),('playfulness','skepticism')


In [217]:
df.columns

Index([u'a', u'b', u'c', u'd', u'e', u'a1', u'a2', u'b1', u'b2', u'c1', u'c2',
       u'd1', u'd2', u'e1', u'e2'],
      dtype='object')

In [224]:
df.to_csv('/Users/Elyssa/Desktop/state_chunks.csv',sep=',')